In [1]:
import pandas as pd
import os
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.regularizers import l2

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1724230379558498448
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 9713889063787891886
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


2024-02-01 12:00:56.451849: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-01 12:00:56.451875: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-01 12:00:56.451911: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-01 12:00:56.451952: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-01 12:00:56.451969: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Reshape, Dense, Dropout, GlobalAveragePooling2D, Input, Flatten, Conv2D,SeparableConv2D, BatchNormalization, Activation, MaxPooling2D, MaxPool2D, GlobalMaxPooling2D,Layer
from tensorflow.keras.optimizers.legacy import Adam,RMSprop
from sklearn.model_selection   import train_test_split
from  tensorflow.keras.utils   import plot_model
import tqdm
from tensorflow.keras import initializers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, Resizing
from tensorflow.keras.layers import UpSampling2D
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [6]:
# Load landmarks from CSV file
def load_landmarks(csv_file):
    data = pd.read_csv(csv_file)
    landmarks = data.values
    return landmarks

In [7]:
def data_generator(directory, batch_size=64, shuffle=True):
    file_paths = []
    labels = []
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        for file in os.listdir(class_dir):
            file_paths.append(os.path.join(class_dir, file))
            labels.append(class_name)
    
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    num_classes = len(np.unique(encoded_labels))

    scaler = MinMaxScaler(feature_range=(0, 1))  # Initialize the MinMaxScaler

    while True:
        if shuffle:
            indices = np.random.permutation(len(file_paths))
        else:
            indices = np.arange(len(file_paths))
        
        for i in range(0, len(indices), batch_size):
            batch_indices = indices[i:i + batch_size]
            batch_input = []
            batch_output = []

            for idx in batch_indices:
                landmarks = load_landmarks(file_paths[idx])
                landmarks_scaled = scaler.fit_transform(landmarks)
                batch_input.append(landmarks_scaled)
                batch_output.append(encoded_labels[idx])

            batch_x = np.array(batch_input)
            batch_y = to_categorical(batch_output, num_classes=num_classes)

            yield batch_x, batch_y


In [8]:
# def data_generator(directory, batch_size=64, shuffle=True):
#     file_paths = []
#     labels = []
#     for class_name in os.listdir(directory):
#         class_dir = os.path.join(directory, class_name)
#         for file in os.listdir(class_dir):
#             file_paths.append(os.path.join(class_dir, file))
#             labels.append(class_name)
    
#     label_encoder = LabelEncoder()
#     encoded_labels = label_encoder.fit_transform(labels)
#     num_classes = len(np.unique(encoded_labels))

#     scaler = MinMaxScaler(feature_range=(0,1))  # Initialize the MinMaxScaler

#     while True:
#         if shuffle:
#             indices = np.random.permutation(len(file_paths))
#         else:
#             indices = np.arange(len(file_paths))
        
#         for i in range(0, len(indices), batch_size):
#             batch_indices = indices[i:i+batch_size]
#             batch_input = []
#             batch_output = []

#             for idx in batch_indices:
#                 landmarks = load_landmarks(file_paths[idx])
#                 # Scale landmarks using MinMaxScaler
#                 landmarks_scaled = scaler.fit_transform(landmarks)
#                 batch_input.append(landmarks_scaled)
#                 batch_output.append(encoded_labels[idx])

#             batch_x = np.array(batch_input)
#             batch_y = to_categorical(batch_output, num_classes=num_classes)

#             yield batch_x, batch_y


In [9]:

# Count samples in each directory
def count_samples(directory):
    count = 0
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            count += len([f for f in os.listdir(class_dir) if f.endswith('.csv')])
    return count

In [10]:
# Define paths
train_dir = "/Users/sanketdhameliya/Documents/CLBPSepDec2024---/ZulfiquarBhottoCLBPCodes/FER+_CSV_data/TRAIN"
valid_dir = "/Users/sanketdhameliya/Documents/CLBPSepDec2024---/ZulfiquarBhottoCLBPCodes/FER+_CSV_data/VALID"
test_dir = "/Users/sanketdhameliya/Documents/CLBPSepDec2024---/ZulfiquarBhottoCLBPCodes/FER+_CSV_data/VALID"

# Create data generators
train_gen = data_generator(train_dir, batch_size=64, shuffle=True)  # Shuffle for training data
valid_gen = data_generator(valid_dir, batch_size=64, shuffle=False)  # No shuffle for validation data
test_gen = data_generator(test_dir, batch_size=64, shuffle=False)

# Count the number of samples
train_samples = count_samples(train_dir)
validation_samples = count_samples(valid_dir)
test_samples = count_samples(test_dir)

# Batch size and steps calculation
batch_size = 64
steps_per_epoch = np.ceil(train_samples / batch_size)
validation_steps = np.ceil(validation_samples / batch_size)
test_steps = np.ceil(test_samples / batch_size)


In [11]:
# inputs = keras.Input(shape=(31, 3, 1), name="img")
# # x = ZulfiquarAugmentationLayer(axis=1, q=3)(inputs)
# x = layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), padding='same',kernel_initializer="he_normal",name="block1_conv1")(inputs)
# x = layers.BatchNormalization()(x)
# x = layers.Activation('relu')(x)
# #x = ZulfiquarFFTLayer()(x)
# x = layers.UpSampling2D(size=(2, 2), name="upsample1")(x)
# x = layers.Conv2D(64, kernel_size=(3,3), padding='same',kernel_initializer="he_normal",name='block1_conv2')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Activation('relu')(x)
# #x = ZulfiquarReLuLayer()(x)  
# x = layers.MaxPooling2D(2,2,name="maxpool_1")(x)


# x = layers.Conv2D(96, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block2_conv1")(x)
# x = layers.BatchNormalization()(x)
# x = layers.Activation('relu')(x)
# x = layers.UpSampling2D(size=(2, 2), name="upsample_after_relu2")(x) 
# #x = ZulfiquarReLuLayer()(x)
# x = layers.Conv2D(96, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block2_conv2")(x)
# x = layers.BatchNormalization()(x)
# x = layers.Activation('relu')(x)
# #x = ZulfiquarReLuLayer()(x)
# # x = layers.Conv2D(96, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block2_conv3")(x)
# # x = layers.BatchNormalization()(x)
# # x = layers.Activation('relu')(x)
# # #x = ZulfiquarReLuLayer()(x) 
# x = layers.MaxPool2D(2,2, name="maxpool_2")(x)
# #x = ZulfiquarAugmentationLayer()(x)
                                                                                                              
# x = layers.Conv2D(128, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block3_conv1")(x)
# x = layers.BatchNormalization()(x)
# x = layers.Activation('relu')(x)
# x = layers.UpSampling2D(size=(2, 2), name="upsample_after_relu3")(x)
# #x = ZulfiquarReLuLayer()(x)
# x = layers.Conv2D(128, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block3_conv2")(x)
# x = layers.BatchNormalization()(x)
# x = layers.Activation('relu')(x)
# #x = ZulfiquarReLuLayer()(x)
# # x = layers.Conv2D(128, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block3_conv3")(x)
# # x = layers.BatchNormalization()(x)
# # x = layers.Activation('relu')(x)
# x = layers.MaxPooling2D(1,1,name="maxpool_3")(x)

# x = layers.GlobalAveragePooling2D(name='GAP')(x)

# #model.add(Flatten())
# output = layers.Dense(7, activation='softmax', name = 'output')(x)

# model = keras.Model(inputs, output, name="toy_resnet")
# model.summary()

In [13]:
inputs = keras.Input(shape=(36, 2, 1), name="landmarks")
#x = ZulfiquarAugmentationLayer(axis=1)(inputs)
x = layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), padding='same',kernel_initializer="he_normal",name="block1_conv1")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
# x = layers.Conv2D(64, kernel_size=(3,3), padding='same',kernel_initializer="he_normal",name='block1_conv2')(x)
# x = layers.BatchNormalization()(x)
# x = layers.Activation('relu')(x)  
x = layers.MaxPooling2D(2,2,name="maxpool_1")(x)
x = ZulfiquarAugmentationLayer(axis=1, q=2)(x)

x = layers.Conv2D(96, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block2_conv1")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.UpSampling2D(size=(2, 2), name="upsample_2")(x)
x = layers.Conv2D(96, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block2_conv2")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(96, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block2_conv3")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x) 
x = layers.MaxPool2D(2,2, name="maxpool_2")(x)
#x = ZulfiquarAugmentationLayer()(x)
                                                                                                              

x = layers.Conv2D(128, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block3_conv1")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.UpSampling2D(size=(2, 2), name="upsample_3")(x)
x = layers.Conv2D(128, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block3_conv2")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.Conv2D(128, kernel_size=(3,3), padding="same", kernel_initializer="he_normal", name="block3_conv3")(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(2,2,name="maxpool_3")(x)

x = layers.GlobalAveragePooling2D(name='GAP')(x)

#model.add(Flatten())
output = layers.Dense(7, activation='softmax', name = 'output')(x)

model = keras.Model(inputs, output, name="toy_resnet")
model.summary()

Model: "toy_resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 landmarks (InputLayer)      [(None, 36, 2, 1)]        0         
                                                                 
 block1_conv1 (Conv2D)       (None, 36, 2, 64)         640       
                                                                 
 batch_normalization_1 (Bat  (None, 36, 2, 64)         256       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 36, 2, 64)         0         
                                                                 
 maxpool_1 (MaxPooling2D)    (None, 18, 1, 64)         0         
                                                                 
 zulfiquar_augmentation_lay  (None, 18, 2, 64)         0         
 er_1 (ZulfiquarAugmentatio                             

In [17]:
model.compile(
    #loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    loss=['categorical_crossentropy'],
    #optimizer=keras.optimizers.RMSprop(),
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
    metrics=["accuracy"],
)
model_name = dump_path+"fer_restnet1_weights.tf"
checkpoint = ModelCheckpoint(model_name, monitor = 'val_accuracy', verbose = 1, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]


In [16]:
# Fit the model
history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=100,
    validation_data=valid_gen,
    validation_steps=validation_steps,
    callbacks=[callbacks_list],
    verbose=2
)

Epoch 1/100


2024-02-01 12:01:05.345494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [ ]:
# # Save history
# np.save(dump_path + 'fer_restnet1_history.npy', history.history)


In [ ]:
# import matplotlib.pyplot as plt


In [ ]:
# plt.figure(figsize=(14, 6))

# plt.subplot(1, 2, 1)
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model Accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='lower right')

# plt.subplot(1, 2, 2)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper right')

# plt.show()

In [ ]:
# test_loss, test_accuracy = model.evaluate(test_gen, steps=test_steps)
# print("Test accuracy:", test_accuracy)

In [ ]:
# validation_steps = int(np.ceil(validation_samples / batch_size))


In [ ]:
# true_labels = []
# predictions = []

# # Ensure validation_steps is an integer
# for _ in range(validation_steps):
#     X_batch, y_batch = next(valid_gen)
#     true_labels.extend(np.argmax(y_batch, axis=1))  # Store true labels
#     preds = model.predict(X_batch)
#     predictions.extend(np.argmax(preds, axis=1))  # Store predictions

# # Convert lists to numpy arrays for further analysis
# true_labels = np.array(true_labels)
# predictions = np.array(predictions)


In [ ]:
# def prepare_data_and_encoder(directory, batch_size=64):
#     file_paths = []
#     labels = []
#     for class_name in os.listdir(directory):
#         class_dir = os.path.join(directory, class_name)
#         for file in os.listdir(class_dir):
#             file_paths.append(os.path.join(class_dir, file))
#             labels.append(class_name)
    
#     label_encoder = LabelEncoder()
#     label_encoder.fit(labels)  # Fit the encoder with all your class labels
#     # Additional logic to prepare your data generator...

#     return label_encoder  # Return the fitted label_encoder

# # Use this function to get your label_encoder
# label_encoder = prepare_data_and_encoder(train_dir, batch_size=64)
# class_names = label_encoder.classes_


In [ ]:
# from sklearn.metrics import confusion_matrix, classification_report
# import seaborn as sns
# import matplotlib.pyplot as plt

# from sklearn.preprocessing import LabelEncoder
# # Assuming true_labels and predictions are already collected from the validation generator
# cm = confusion_matrix(true_labels, predictions)

# # Plotting the confusion matrix
# plt.figure(figsize=(10, 8))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.show()

# # Printing the classification report
# print(classification_report(true_labels, predictions, target_names=class_names))


In [ ]:
# import matplotlib.pyplot as plt

# def plot_learning_curves(history):
#     acc = history.history['accuracy']
#     val_acc = history.history['val_accuracy']
#     loss = history.history['loss']
#     val_loss = history.history['val_loss']
#     epochs = range(1, len(acc) + 1)

#     plt.figure(figsize=(12, 5))
#     plt.subplot(1, 2, 1)
#     plt.plot(epochs, acc, 'bo', label='Training acc')
#     plt.plot(epochs, val_acc, 'b', label='Validation acc')
#     plt.title('Training and validation accuracy')
#     plt.legend()

#     plt.subplot(1, 2, 2)
#     plt.plot(epochs, loss, 'bo', label='Training loss')
#     plt.plot(epochs, val_loss, 'b', label='Validation loss')
#     plt.title('Training and validation loss')
#     plt.legend()

#     plt.show()


In [ ]:
# plot_learning_curves(history)

In [ ]:
# model = load_model(model_name)
# results = model.evaluate(xag_test,y_test, verbose=2)
# print(results)
# print('Categorical accuracy, saved model {}%!'.format(
#     round(results[1]*100,2)))
# print('Crossentropy loss, saved model {}!'.format(
#     round(results[0],2)))